In [11]:
import networkx as nx
import numpy as np

def leiden_algorithm(G, P, gamma=1):
    """
    Implements the Leiden algorithm for community detection on a graph G with initial partition P.

    Args:
        G: A NetworkX graph object.
        P: A NumPy array of integers representing the initial partition of the nodes of G.
        gamma: The resolution parameter for the Leiden algorithm.

    Returns:
        A NumPy array of integers representing the final partition of the nodes of G.
    """
    # Helper function to calculate the modularity gain of moving a node to a new community
    def delta_Q(G, P, node, new_community):
        k_in = np.sum([G[node][neighbor]['weight'] for neighbor in G.neighbors(node) if P[neighbor] == new_community])
        k_out = np.sum([G[node][neighbor]['weight'] for neighbor in G.neighbors(node)]) - k_in
        q_in = k_in / G.size(weight='weight')
        q_out = k_out / G.size(weight='weight')
        k = np.sum([G.degree(node, weight='weight') for node in G.nodes()])
        delta = 2 * (q_in - q_out) - gamma * ((2 * k_in + G.degree(node, weight='weight')) / (2 * k) - (k_in + k_out) / k) ** 2
        return delta

    # Helper function to move a node to a new community and update the partition
    def move_node(G, P, node, new_community):
        old_community = P[node]
        P[node] = new_community
        for neighbor in G.neighbors(node):
            if P[neighbor] == old_community:
                P = move_node(G, P, neighbor, new_community)
        return P

    done = False
    while not done:
        P = move_nodes_fast(G, P, delta_Q)
        if not np.array_equal(P, np.arange(G.number_of_nodes())):
            prefined = refine_partition(G, P, delta_Q)
            G = aggregate_graph(G, prefined)
            P = np.array([prefined.index(P[c]) for c in G.nodes()])
        else:
            done = True

    return flat(P)

def move_nodes_fast(G, P, delta_Q):
    Q = list(G.nodes())
    while Q:
        node = Q.pop(0)
        community = P[node]
        new_community = max(set(P).union({-1}), key=lambda c: delta_Q(G, P, node, c))
        if new_community != community:
            P = move_node(G, P, node, new_community)
            neighbors = [neighbor for neighbor in G.neighbors(node) if neighbor not in Q]
            Q += neighbors
    return P

def refine_partition(G, P, delta_Q):
    prefined = np.arange(G.number_of_nodes())
    for community in np.unique(P):
        subset = np.where(P == community)[0]
        prefined = merge_nodes_subset(G, prefined, subset, delta_Q)
    return prefinedimport networkx as nx
import numpy as np

def leiden_algorithm(G, P, gamma=1):
    """
    Implements the Leiden algorithm for community detection on a graph G with initial partition P.

    Args:
        G: A NetworkX graph object.
        P: A NumPy array of integers representing the initial partition of the nodes of G.
        gamma: The resolution parameter for the Leiden algorithm.

    Returns:
        A NumPy array of integers representing the final partition of the nodes of G.
    """
    # Helper function to calculate the modularity gain of moving a node to a new community
    def delta_Q(G, P, node, new_community):
        k_in = np.sum([G[node][neighbor]['weight'] for neighbor in G.neighbors(node) if P[neighbor] == new_community])
        k_out = np.sum([G[node][neighbor]['weight'] for neighbor in G.neighbors(node)]) - k_in
        q_in = k_in / G.size(weight='weight')
        q_out = k_out / G.size(weight='weight')
        k = np.sum([G.degree(node, weight='weight') for node in G.nodes()])
        delta = 2 * (q_in - q_out) - gamma * ((2 * k_in + G.degree(node, weight='weight')) / (2 * k) - (k_in + k_out) / k) ** 2
        return delta

    # Helper function to move a node to a new community and update the partition
    def move_node(G, P, node, new_community):
        old_community = P[node]
        P[node] = new_community
        for neighbor in G.neighbors(node):
            if P[neighbor] == old_community:
                P = move_node(G, P, neighbor, new_community)
        return P

    done = False
    while not done:
        P = move_nodes_fast(G, P, delta_Q)
        if not np.array_equal(P, np.arange(G.number_of_nodes())):
            prefined = refine_partition(G, P, delta_Q)
            G = aggregate_graph(G, prefined)
            P = np.array([prefined.index(P[c]) for c in G.nodes()])
        else:
            done = True

    return flat(P)

def move_nodes_fast(G, P, delta_Q):
    Q = list(G.nodes())
    while Q:
        node = Q.pop(0)
        community = P[node]
        new_community = max(set(P).union({-1}), key=lambda c: delta_Q(G, P, node, c))
        if new_community != community:
            P = move_node(G, P, node, new_community)
            neighbors = [neighbor for neighbor in G.neighbors(node) if neighbor not in Q]
            Q += neighbors
    return P

def refine_partition(G, P, delta_Q):
    prefined = np.arange(G.number_of_nodes())
    for community in np.unique(P):
        subset = np.where(P == community)[0]
        prefined = merge_nodes_subset(G, prefined, subset, delta_Q)
    return prefined

In [15]:
H = nx.read_gml("DataSets/réels/football/football.gml")
start = timeit.default_timer()
# leidan 
end = timeit.default_timer()
print("Time",end - start)

Time 2.1500000002561137e-05
